In [3]:
#Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline

## Data Preprocessing

RNN input requires an array data type. The `full_review_text` column will be transformed into the `X` array and the “sentiment” column into the `y` array.

In [8]:
#Import Data
df = pd.read_csv('../data/housing_data_encoded.csv')
#test = pd.read_csv("/data/Housing/test.csv")
df.head()

,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,State_TX,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
0,2,1,67653.0,133.0,0,1,0,0,1,1,1,0
1,2,2,89995.0,133.0,0,1,0,0,1,1,1,0
2,2,3,116641.0,133.0,0,1,0,0,1,1,1,0
3,2,4,167849.0,133.0,0,1,0,0,1,1,1,0
4,2,5,303323.0,133.0,0,1,0,0,1,1,1,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 12 columns):
Identifier                                1220 non-null int64
Bedroom                                   1220 non-null int64
Price                                     1151 non-null float64
Average_Market_Days                       1030 non-null float64
RegionName_Crosby                         1220 non-null int64
RegionName_Houston                        1220 non-null int64
RegionName_Pasadena                       1220 non-null int64
RegionName_Porter                         1220 non-null int64
State_TX                                  1220 non-null int64
Metro_Houston-The Woodlands-Sugar Land    1220 non-null int64
CountyName_Harris County                  1220 non-null int64
CountyName_Montgomery County              1220 non-null int64
dtypes: float64(2), int64(10)
memory usage: 114.5 KB


In [11]:
# drop N/A
housing_df=df.dropna()

In [12]:
# Creating the X and y sets
#X = housing_df.drop(columns=["Price"]).values
#y = housing_df["Price"].values

In [7]:
#slice data into Training and Testing
train = df.loc['2010':'2019']

test = df.loc[:'2020']
test

,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,State_TX,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
0,2,1,67653.0,133.0,0,1,0,0,1,1,1,0
1,2,2,89995.0,133.0,0,1,0,0,1,1,1,0
2,2,3,116641.0,133.0,0,1,0,0,1,1,1,0
3,2,4,167849.0,133.0,0,1,0,0,1,1,1,0
4,2,5,303323.0,133.0,0,1,0,0,1,1,1,0
5,223,1,53098.0,132.0,0,0,1,0,1,1,1,0
6,223,2,71091.0,132.0,0,0,1,0,1,1,1,0
7,223,3,102976.0,132.0,0,0,1,0,1,1,1,0
8,1811,2,NaN,NaN,0,0,0,1,1,1,0,1
9,2134,2,84980.0,NaN,1,0,0,0,1,1,1,0


In [11]:
#importing function
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def factorize(data, var, fill_na = None):
      if fill_na is not None:
            data[var].fillna(fill_na, inplace=True)
      le.fit(data[var])
      data[var] = le.transform(data[var])
      return data

Now that the data is encoded, the training and testing sets will be created.

In [9]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, random_state=78)

In [ ]:
# Import Keras modules for data encoding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

 ### Making Predictions